## LSTM CNN BIDIRECTIONAL

In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Bidirectional
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [3]:
import pickle

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

Embeding

In [5]:
max_features = 20000
maxlen = 20
embedding_size = 128

Convolution

In [6]:
kernel_size = 5
filters = 64
pool_size = 4

LSTM

In [7]:
lstm_output_size = 128

Training

In [8]:
batch_size = 32
epochs = 20

In [9]:
def load_ds(path):
    vecfile = open(path,'r')
    return pickle.load(vecfile)

In [10]:
ds = load_ds('./data/ann/vectors_50_50.txt')
ds.head()

,category,tweet_sec
0,1,"[268, 459, 146, 2, 470, 4493]"
1,1,"[2264, 7, 9784, 3050]"
2,0,"[80, 172, 35, 1, 14, 39, 2265, 7, 9785, 2266, ..."
3,1,"[107, 41, 21, 471, 9, 659, 5, 216, 300, 5, 7, ..."
4,1,"[562, 106, 29, 563, 29, 422]"


In [11]:
print('Pad sequences (samples x time)')
tweet_sec = sequence.pad_sequences(ds['tweet_sec'], maxlen=maxlen)
print('tweet shape:', tweet_sec.shape)

Pad sequences (samples x time)
('tweet shape:', (15306, 20))


In [12]:
def build_arch():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Bidirectional(
                LSTM(lstm_output_size,
                   activation='tanh', 
                   recurrent_activation='hard_sigmoid', 
                   dropout=0.2, 
                   recurrent_dropout=0.2
                    )
                )
             )
    model.add(Dense(1, activation='sigmoid'))
    return model

In [13]:
def cross_val(tweet_sec, target, nFolds):
    kFold = StratifiedKFold(n_splits=nFolds, shuffle=True)
    scores = []
    for train, test in kFold.split(tweet_sec,target):
        model = build_arch()
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(tweet_sec[train],
                  target[train],
                  batch_size=batch_size,
                  epochs=epochs,verbose=1,
                  validation_split=0.1)
        score, acc = model.evaluate(tweet_sec[test], target[test],batch_size=batch_size)
        scores.append(acc * 100)
    return scores   

In [14]:
#Training with the 100% of the data set
def train_model(tweet_sec,target):
    model = build_arch()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model.fit(tweet_sec, target,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)        

In [15]:
results = cross_val(tweet_sec,ds['category'],10)

Build model...
Train on 12396 samples, validate on 1378 samples
Epoch 1/20
12396/12396 [==============================] - 25s - loss: 0.4010 - acc: 0.8145 - val_loss: 0.3048 - val_acc: 0.8803
Epoch 2/20
12396/12396 [==============================] - 25s - loss: 0.1843 - acc: 0.9329 - val_loss: 0.2955 - val_acc: 0.8882
Epoch 3/20
12396/12396 [==============================] - 23s - loss: 0.0873 - acc: 0.9707 - val_loss: 0.3664 - val_acc: 0.8578
Epoch 4/20
12396/12396 [==============================] - 23s - loss: 0.0382 - acc: 0.9888 - val_loss: 0.5516 - val_acc: 0.8549
Epoch 5/20
12396/12396 [==============================] - 23s - loss: 0.0207 - acc: 0.9939 - val_loss: 0.5192 - val_acc: 0.8672
Epoch 6/20
12396/12396 [==============================] - 23s - loss: 0.0149 - acc: 0.9956 - val_loss: 0.6437 - val_acc: 0.8534
Epoch 7/20
12396/12396 [==============================] - 23s - loss: 0.0109 - acc: 0.9965 - val_loss: 0.7875 - val_acc: 0.8578
Epoch 8/20
12396/12396 [================

12398/12398 [==============================] - 24s - loss: 0.3999 - acc: 0.8134 - val_loss: 0.2873 - val_acc: 0.8824
Epoch 2/20
12398/12398 [==============================] - 23s - loss: 0.1832 - acc: 0.9291 - val_loss: 0.3480 - val_acc: 0.8672
Epoch 3/20
12398/12398 [==============================] - 23s - loss: 0.0828 - acc: 0.9721 - val_loss: 0.4222 - val_acc: 0.8491
Epoch 4/20
12398/12398 [==============================] - 23s - loss: 0.0379 - acc: 0.9871 - val_loss: 0.5403 - val_acc: 0.8541
Epoch 5/20
12398/12398 [==============================] - 23s - loss: 0.0186 - acc: 0.9937 - val_loss: 0.6922 - val_acc: 0.8425
Epoch 6/20
12398/12398 [==============================] - 23s - loss: 0.0133 - acc: 0.9956 - val_loss: 0.7083 - val_acc: 0.8476
Epoch 7/20
12398/12398 [==============================] - 23s - loss: 0.0107 - acc: 0.9961 - val_loss: 0.9183 - val_acc: 0.8483
Epoch 8/20
12398/12398 [==============================] - 23s - loss: 0.0086 - acc: 0.9966 - val_loss: 0.8411 - val

12398/12398 [==============================] - 23s - loss: 0.1830 - acc: 0.9321 - val_loss: 0.2994 - val_acc: 0.8716
Epoch 3/20
12398/12398 [==============================] - 23s - loss: 0.0784 - acc: 0.9738 - val_loss: 0.4816 - val_acc: 0.8512
Epoch 4/20
12398/12398 [==============================] - 23s - loss: 0.0342 - acc: 0.9894 - val_loss: 0.5464 - val_acc: 0.8462
Epoch 5/20
12398/12398 [==============================] - 23s - loss: 0.0186 - acc: 0.9940 - val_loss: 0.5823 - val_acc: 0.8454
Epoch 6/20
12398/12398 [==============================] - 23s - loss: 0.0110 - acc: 0.9964 - val_loss: 0.7339 - val_acc: 0.8527
Epoch 7/20
12398/12398 [==============================] - 23s - loss: 0.0097 - acc: 0.9968 - val_loss: 0.9143 - val_acc: 0.8345
Epoch 8/20
12398/12398 [==============================] - 23s - loss: 0.0088 - acc: 0.9969 - val_loss: 0.9416 - val_acc: 0.8491
Epoch 9/20
12398/12398 [==============================] - 23s - loss: 0.0085 - acc: 0.9972 - val_loss: 0.7976 - val

12398/12398 [==============================] - 22s - loss: 0.0794 - acc: 0.9727 - val_loss: 0.5160 - val_acc: 0.8563
Epoch 4/20
12398/12398 [==============================] - 23s - loss: 0.0401 - acc: 0.9880 - val_loss: 0.5355 - val_acc: 0.8476
Epoch 5/20
12398/12398 [==============================] - 22s - loss: 0.0200 - acc: 0.9940 - val_loss: 0.6561 - val_acc: 0.8498
Epoch 6/20
12398/12398 [==============================] - 23s - loss: 0.0145 - acc: 0.9951 - val_loss: 0.8066 - val_acc: 0.8316
Epoch 7/20
12398/12398 [==============================] - 23s - loss: 0.0101 - acc: 0.9971 - val_loss: 0.9148 - val_acc: 0.8454
Epoch 8/20
12398/12398 [==============================] - 23s - loss: 0.0082 - acc: 0.9975 - val_loss: 1.0241 - val_acc: 0.8374
Epoch 9/20
12398/12398 [==============================] - 23s - loss: 0.0093 - acc: 0.9971 - val_loss: 0.8337 - val_acc: 0.8483
Epoch 10/20
12398/12398 [==============================] - 23s - loss: 0.0058 - acc: 0.9978 - val_loss: 1.0116 - va

In [16]:
print("Acc: %.2f (+/- %.2f)" %(np.mean(results), np.std(results)))

Acc: 84.29 (+/- 1.28)


In [17]:
for i in range (0,len(results)):
    print "Model %d, acc: %.2f " %(i+1,results[i])

Model 1, acc: 83.22 
Model 2, acc: 85.51 
Model 3, acc: 84.73 
Model 4, acc: 83.20 
Model 5, acc: 85.10 
Model 6, acc: 85.23 
Model 7, acc: 81.44 
Model 8, acc: 83.92 
Model 9, acc: 85.88 
Model 10, acc: 84.64 


In [18]:
trained_model = train_model(tweet_sec,ds['category'])

Build model...
Epoch 1/20
15306/15306 [==============================] - 29s - loss: 0.3740 - acc: 0.8325    
Epoch 2/20
15306/15306 [==============================] - 27s - loss: 0.1787 - acc: 0.9326    
Epoch 3/20
15306/15306 [==============================] - 28s - loss: 0.0830 - acc: 0.9706    
Epoch 4/20
15306/15306 [==============================] - 28s - loss: 0.0402 - acc: 0.9869    
Epoch 5/20
15306/15306 [==============================] - 28s - loss: 0.0252 - acc: 0.9910    
Epoch 6/20
15306/15306 [==============================] - 28s - loss: 0.0150 - acc: 0.9950    
Epoch 7/20
15306/15306 [==============================] - 27s - loss: 0.0108 - acc: 0.9965    
Epoch 8/20
15306/15306 [==============================] - 28s - loss: 0.0085 - acc: 0.9975    
Epoch 9/20
15306/15306 [==============================] - 27s - loss: 0.0097 - acc: 0.9962    
Epoch 10/20
15306/15306 [==============================] - 27s - loss: 0.0077 - acc: 0.9972    
Epoch 11/20
15306/15306 [=========

In [19]:
model_json = trained_model.model.to_json()
with open("./data/models/ccn_bi_lstm/bi_lstm_cnn.json",'w') as json_file:
    json_file.write(model_json)
trained_model.model.save_weights("./data/models/ccn_bi_lstm/bi_lstm_cnn_weights.h5")

In [20]:
trained_model.model.save("./data/models/ccn_bi_lstm/bi_lstm_cnn.h5")